In [1]:
import pandas as pd
import matplotlib as plt
import rasterio
import numpy as np

In [2]:
df = pd.read_csv("C:/Users/User/PycharmProjects/SafeRoute/NYPD_Arrests_Data__Historic_.csv")
df = df.replace('(null)', np.nan)
df = df.replace('nan', np.nan)
df = df.dropna(axis=0)
df

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
1,237354740,12/04/2021,153.0,RAPE 3,104.0,RAPE,PL 1302502,F,B,41,0.0,25-44,M,WHITE HISPANIC,1013232.0,236725.0,40.816392,-73.895296,POINT (-73.89529641399997 40.816391847000034)
2,236081433,11/09/2021,681.0,"CHILD, ENDANGERING WELFARE",233.0,SEX CRIMES,PL 2601001,M,Q,113,0.0,25-44,M,BLACK,1046367.0,186986.0,40.679700,-73.776047,POINT (-73.77604736799998 40.67970040800003)
4,192799737,01/26/2019,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,0.0,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045)
7,236106641,11/10/2021,263.0,"ARSON 2,3,4",114.0,ARSON,PL 1501001,F,B,41,72.0,25-44,M,WHITE HISPANIC,1017934.0,232221.0,40.804013,-73.878332,POINT (-73.87833183299993 40.804012949000025)
8,238383628,12/28/2021,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1702500,F,Q,113,0.0,18-24,M,BLACK,1045482.0,191341.0,40.691660,-73.779199,POINT (-73.77919852099996 40.69166001700007)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5498645,255851599,12/13/2022,969.0,"TRAFFIC,UNCLASSIFIED INFRACTIO",881.0,OTHER TRAFFIC INFRACTION,VTL051101A,M,M,32,0.0,25-44,M,BLACK,1000188.0,238820.0,40.822167,-73.942415,POINT (-73.94241483 40.82216691)
5498646,256469262,12/16/2022,198.0,CRIMINAL CONTEMPT 1,126.0,MISCELLANEOUS PENAL LAW,PL 21551B3,F,B,46,0.0,25-44,M,BLACK,1011750.0,250274.0,40.853578,-73.900591,POINT (-73.900591 40.853578)
5498647,260305039,12/20/2022,548.0,"GAMBLING, DEVICE, POSSESSION",350.0,GAMBLING,PL 2253001,M,K,72,0.0,25-44,F,ASIAN / PACIFIC ISLANDER,981944.0,171869.0,40.638420,-74.008307,POINT (-74.008307 40.63842)
5498648,255843041,12/13/2022,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,5,0.0,45-64,M,ASIAN / PACIFIC ISLANDER,986159.0,200104.0,40.715917,-73.993111,POINT (-73.993111 40.715917)


In [3]:
# np.unique(np.array())
list(set(df['OFNS_DESC']))

['DISRUPTION OF A RELIGIOUS SERV',
 'DANGEROUS DRUGS',
 'OFF. AGNST PUB ORD SENSBLTY &',
 'FORTUNE TELLING',
 'BURGLARY',
 "BURGLAR'S TOOLS",
 'MISCELLANEOUS PENAL LAW',
 'UNAUTHORIZED USE OF A VEHICLE',
 'CANNABIS RELATED OFFENSES',
 'OTHER STATE LAWS',
 'DANGEROUS WEAPONS',
 'OFF. AGNST PUB ORD SENSBLTY & RGHTS TO PRIV',
 'FELONY ASSAULT',
 'ABORTION',
 'CHILD ABANDONMENT/NON SUPPORT',
 'OFFENSES AGAINST THE PERSON',
 'POSSESSION OF STOLEN PROPERTY 5',
 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED',
 'ADMINISTRATIVE CODE',
 'OTHER TRAFFIC INFRACTION',
 'HOMICIDE-NEGLIGENT-VEHICLE',
 'SEX CRIMES',
 'PARKING OFFENSES',
 'UNAUTHORIZED USE OF A VEHICLE 3 (UUV)',
 'OFFENSES INVOLVING FRAUD',
 'GRAND LARCENY',
 'ENDAN WELFARE INCOMP',
 'RAPE',
 'GRAND LARCENY OF MOTOR VEHICLE',
 'OTHER STATE LAWS (NON PENAL LA',
 'F.C.A. P.I.N.O.S.',
 'OFFENSES AGAINST MARRIAGE UNCLASSIFIED',
 'ROBBERY',
 'DISRUPTION OF A RELIGIOUS SERVICE',
 'THEFT OF SERVICES',
 'OFFENSES AGAINST PUBLIC ADMINISTRATION',
 'OTHER

In [4]:
danger_scores = {
    'HOMICIDE-NEGLIGENT,UNCLASSIFIED': 10,
    'SEX CRIMES': 9,
    'GRAND LARCENY': 3,
    'ROBBERY': 8,
    'UNLAWFUL POSS. WEAP. ON SCHOOL GROUNDS': 6,
    'KIDNAPPING & RELATED OFFENSES': 9,
    'BURGLARY': 6,
    'DANGEROUS DRUGS': 5,
    'FRAUDS': 2,
    'RAPE': 12,
    'FELONY SEX CRIMES': 10,
    'ASSAULT 3 & RELATED OFFENSES': 8,
    'UNAUTHORIZED USE OF A VEHICLE 3 (UUV)': 4,
    'FELONY ASSAULT': 9,
    'MURDER & NON-NEGL. MANSLAUGHTER': 15,
    'THEFT-FRAUD': 3,
    'PETIT LARCENY': 2
}

In [5]:
new_df = df.loc[df['OFNS_DESC'].isin(danger_scores.keys()), ['ARREST_DATE', 'Latitude', 'Longitude']]
new_df

,ARREST_DATE,Latitude,Longitude
1,12/04/2021,40.816392,-73.895296
2,11/09/2021,40.679700,-73.776047
4,01/26/2019,40.800694,-73.941109
9,01/06/2016,40.648650,-73.950336
10,12/04/2021,40.688584,-73.916526
...,...,...,...
5498636,12/11/2022,40.637301,-74.009474
5498639,12/30/2022,40.671104,-73.913562
5498641,12/13/2022,40.742954,-73.955274
5498648,12/13/2022,40.715917,-73.993111


In [6]:
new_df['Danger Score'] = df.loc[df['OFNS_DESC'].isin(danger_scores.keys()), 'OFNS_DESC'].map(danger_scores)
new_df

,ARREST_DATE,Latitude,Longitude,Danger Score
1,12/04/2021,40.816392,-73.895296,12
2,11/09/2021,40.679700,-73.776047,9
4,01/26/2019,40.800694,-73.941109,9
9,01/06/2016,40.648650,-73.950336,12
10,12/04/2021,40.688584,-73.916526,8
...,...,...,...,...
5498636,12/11/2022,40.637301,-74.009474,5
5498639,12/30/2022,40.671104,-73.913562,8
5498641,12/13/2022,40.742954,-73.955274,9
5498648,12/13/2022,40.715917,-73.993111,8


In [7]:
new_df.sort_values(by=['Longitude'])

,ARREST_DATE,Latitude,Longitude,Danger Score
3031914,03/27/2012,40.507709,-74.254939,5
3015760,03/27/2012,40.507709,-74.254939,5
2948228,06/05/2012,40.508303,-74.254560,8
3287428,08/02/2011,40.508303,-74.254560,9
3291149,07/20/2011,40.508303,-74.254560,5
...,...,...,...,...
3836331,04/23/2010,51.627063,-73.681780,8
3283455,07/30/2011,51.627063,-73.681780,8
3965562,12/11/2009,51.627063,-73.681780,5
4226739,05/22/2009,51.627063,-73.681780,5


In [8]:
(new_df['Latitude'].min(), new_df['Latitude'].max(), new_df['Longitude'].min(), new_df['Longitude'].max())

(40.49890536300006, 62.08307497800007, -74.25493873599999, -73.68178026799995)

In [9]:
with rasterio.open("C:/Users/User/PycharmProjects/SafeRoute/clustered.tif") as src:
    data = src.read()
    image = src.read(1)
    clusters = src.read(2)
    bounds = src.bounds

In [10]:
bounds

BoundingBox(left=-74.27999999999999, bottom=40.400000004, right=-73.66000000247999, top=41.4)

In [11]:
(abs(bounds[2] - bounds[0]), abs(bounds[3] - bounds[1]))

(0.6199999975199972, 0.9999999959999997)

In [12]:
step = (abs(bounds[2] - bounds[0])/np.shape(clusters)[1] + abs(bounds[3] - bounds[1])/np.shape(clusters)[0]) / 2
step

0.000833333329999998

In [13]:
cluster_score = {c: 0 for c in np.unique(clusters)}
cluster_crime_num = {c: 0 for c in np.unique(clusters)}
cluster_score

{-1: 0,
 0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 0,
 62: 0,
 63: 0,
 64: 0,
 65: 0,
 66: 0,
 67: 0,
 68: 0,
 69: 0,
 70: 0,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 0,
 76: 0,
 77: 0,
 78: 0,
 79: 0,
 80: 0,
 81: 0,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 0,
 87: 0,
 88: 0,
 89: 0,
 90: 0,
 91: 0,
 92: 0,
 93: 0,
 94: 0,
 95: 0,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 0,
 101: 0,
 102: 0,
 103: 0,
 104: 0,
 105: 0,
 106: 0,
 107: 0,
 108: 0,
 109: 0,
 110: 0,
 111: 0,
 112: 0,
 113: 0,
 114: 0,
 115: 0,
 116: 0,
 117: 0,
 118: 0,
 119: 0,
 120: 0,
 121: 0,
 122

In [14]:
for row in new_df.itertuples():
    p = (row.Longitude, row.Latitude) # point declaration
    if abs(p[0])>abs(bounds[0]) or abs(p[0])<abs(bounds[2]): 
        continue
    elif abs(p[1])<abs(bounds[1]) or abs(p[1])>abs(bounds[3]): 
        continue 
    cluster_score[clusters[round((abs(p[1]) - abs(bounds[1]))/step) - 1, round((abs(p[0]) - abs(bounds[2]))/step) - 1]] += row._4
    cluster_crime_num[clusters[round((abs(p[1]) - abs(bounds[1]))/step) - 1, round((abs(p[0]) - abs(bounds[2]))/step) - 1]] += 1

In [15]:
cluster_score

{-1: 12977720,
 0: 0,
 1: 0,
 2: 16,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 7345,
 9: 16934,
 10: 0,
 11: 1512,
 12: 40508,
 13: 250806,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 59692,
 19: 0,
 20: 321450,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 186,
 62: 0,
 63: 676,
 64: 0,
 65: 683,
 66: 0,
 67: 2347,
 68: 0,
 69: 0,
 70: 0,
 71: 67,
 72: 133,
 73: 14447,
 74: 13080,
 75: 0,
 76: 0,
 77: 28439,
 78: 0,
 79: 0,
 80: 0,
 81: 76111,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 46411,
 87: 0,
 88: 79270,
 89: 90562,
 90: 0,
 91: 68455,
 92: 46107,
 93: 0,
 94: 117966,
 95: 118335,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 851,
 101: 161456,
 102: 0,
 103: 876,
 104: 20123,
 105: 22124,
 106: 4444,
 107: 359821,
 108: 597,


In [16]:
score_band = []
crime_band = []
cluster_score[-1] = 0
cluster_crime_num[-1] = 0


for i in clusters:
    for j in i:
        score_band.append(cluster_score[j])
        crime_band.append(cluster_crime_num[j])

crime_band = np.array(crime_band).reshape(np.shape(clusters))
score_band = np.array(score_band).reshape(np.shape(clusters))
cluster_crime_num

{-1: 0,
 0: 0,
 1: 0,
 2: 2,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 1243,
 9: 3584,
 10: 0,
 11: 235,
 12: 7385,
 13: 40229,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 11298,
 19: 0,
 20: 54656,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 59: 0,
 60: 0,
 61: 30,
 62: 0,
 63: 107,
 64: 0,
 65: 110,
 66: 0,
 67: 407,
 68: 0,
 69: 0,
 70: 0,
 71: 9,
 72: 22,
 73: 2451,
 74: 2176,
 75: 0,
 76: 0,
 77: 4755,
 78: 0,
 79: 0,
 80: 0,
 81: 13199,
 82: 0,
 83: 0,
 84: 0,
 85: 0,
 86: 8523,
 87: 0,
 88: 13764,
 89: 14769,
 90: 0,
 91: 11802,
 92: 7617,
 93: 0,
 94: 17084,
 95: 18318,
 96: 0,
 97: 0,
 98: 0,
 99: 0,
 100: 135,
 101: 30065,
 102: 0,
 103: 144,
 104: 3525,
 105: 3317,
 106: 681,
 107: 65902,
 108: 102,
 109: 99,
 110: 131,
 111: 0,